<a href="https://colab.research.google.com/github/mthomp89/landslide-detect/blob/main/BIG_SUR_Landslide_Version_of_Detecting_Changes_in_Sentinel_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Integrating Synthetic Aperature Radar Imagery in Landslide Detection</center>
 <table><tr><td>By Leah Manak and Mitchell Thompson</td></tr></table>
  <table><tr><td>Special Credit to Elsa Culler</td></tr></table>
  <table><tr><td>June, 2022</td></tr></table>




# In this Notebook:
 1. Setup
 - 1.1 Initialize Google Earth Engine
 - 1.2 Import Datasets and Python Modules
 - 1.3 Set Working Directory
 2. Verified Landslides: Database Setup and Location Map
 - 2.1 Create Geopandas Dataframe
 - 2.2 Entire Landslides Catalog Folium Map
 - 2.3 Establish AOI
 3. Sentinal-1 Radar Imagery
 - 3.1 Process significant changes
 4. Discussion
 5. References

# Setup

### Initialize Google Earth Engine

Run the following cell to initialize the API. The output will contain instructions on how to grant this notebook access to Earth Engine using your account.

In [1]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AX4XfWhDahZYT7hpUuDFobem566GxEUKS7CFobDHXc7KNs8Wob16aOmAArY

Successfully saved authorization token.


### Datasets and Python modules
One [dataset](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD) that will be used in the tutorial is:

- COPERNICUS/S1_GRD_FLOAT
    - Sentinel-1 ground range detected images

Another is a verified landslide locations dataset created by CU Boulder Earth Lab that will be defined below as "landslide_df". 
- This dataset includes various verified locations of landslides across North America with descriptions of severity and type. 
The following cell imports some python modules which we will be using as we go along and enables inline graphics.

The following cell imports various Python modules necessary to complete this notebook

In [2]:
import os

import json
import earthpy as et
import folium
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import geopandas as gpd
# from geopandas import GeoDataFrame
import shapely.geometry as sgeo
from shapely.geometry import Point
from src.det import det
from src.create_dataframe import create_dataframe
from src.image_search import image_search
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
import geemap.foliumap as geemap
import ipyleaflet
import ipywidgets

# Enable inline graphics
%matplotlib inline

### Set working directory

In [3]:
# Change directory to landslide-detect data path
data_path = os.path.join(et.io.HOME, "earth-analytics", "landslide-detect")
if os.path.exists(data_path):
    os.chdir(data_path)
else:
    os.makedirs(data_path)
    print('The new directory is created!')
    os.chdir(data_path)

print('Current working directory is set to: ', os.getcwd())

Current working directory is set to:  /Users/leahmanak/earth-analytics/landslide-detect


# Verified Landslides: Database Setup and Location Map

### Create Geodataframe

In [4]:
# Open CSV and Create DataFrame with Pandas .. This should be replaced with cell below
landslide_gdf = create_dataframe('landslides.verified.csv')
landslide_gdf.head()

,slide.id,slide.date,pre_event,post_event,location,type,trigger,size,lon,lat,location_accuracy,event_title,admin_division_name,ge.lat,ge.lon,is.exact,geometry,ctr_point
0,8321,2015-09-21,2015-03-25,2016-03-19,"Birken, BC, Canada",landslide,downpour,medium,-122.620500,50.479000,5km,"Birken, BC, Canada",British Columbia,NA,NA,FALSE,POINT (-122.62050 50.47900),"-122.6205, 50.479"
1,7757,2015-12-07,2015-06-10,2016-06-04,Edmonds,mudslide,rain,medium,-122.380528,47.706794,5km,Edmonds,Washington,47.70679444444445,-122.38052777777777,TRUE,POINT (-122.38053 47.70679),"-122.38052777777777, 47.70679444444445"
2,8596,2015-10-04,2015-04-07,2016-04-01,"Between 3rd and 7th Ave, Coast Highway, Laguna...",mudslide,rain,medium,-117.743100,33.500600,exact,"Between 3rd and 7th Ave, Coast Highway, Laguna...",California,NA,NA,FALSE,POINT (-117.74310 33.50060),"-117.7431, 33.5006"
3,8678,2015-10-08,2015-04-11,2016-04-05,between Fort Davis and Fort Davis State Park,rock_fall,flooding,medium,-103.922061,30.602311,exact,between Fort Davis and Fort Davis State Park,Texas,30.602311111111113,-103.92206111111112,TRUE,POINT (-103.92206 30.60231),"-103.92206111111112, 30.602311111111113"
4,9013,2015-12-09,2015-06-12,2016-06-06,I-5 near Portland border,rock_fall,flooding,medium,-122.657300,45.732600,1km,I-5 near Portland border,Washington,NA,NA,FALSE,POINT (-122.65730 45.73260),"-122.6573, 45.7326"


In [5]:
# # A function to create a geodataframe of all verified landslides
# def create_dataframe(data_file):
#     """
#     Reads a csv file into a GeoPandas DataFrame. Sets the index to the 'slide.id'
#     and sorts the geoDataFrame by the new index. Converts the date+time column to
#     a DateTime object and drops the time. Converts the reported geocoordinates to
#     a geoPandas geometry point.

#     Parameters
#     ----------
#     data_file : str

#     Returns
#     -------
#     dataframe
#         sorted by index
#     """

#     gdf = gpd.read_file(data_file, index_col='slide.id').sort_values(by='slide.id')
#     gdf = gdf.set_index('slide.id')
#     # Convert to DatTime object, drop time
#     gdf['slide.date'] = pd.to_datetime(gdf['slide.date'])
#     gdf['slide.date'] = gdf['slide.date'].dt.date
#     # Convert reported geocoordinates to geometery
#     gdf[['lon', 'lat']] = gdf[['lon', 'lat']].astype(float)
#     geometry = [Point(xy) for xy in zip(gdf.lon, gdf.lat)]
#     gdf = gdf.drop(['lon', 'lat'], axis=1)
#     gdf = GeoDataFrame(gdf, crs="EPSG:4326", geometry=geometry)

#     return gdf
# landslide_gdf = create_dataframe('landslides.verified.csv')
# landslide_gdf.head()

### Folium Map of All Verified Landslide Locations

In [6]:
# Display all verified landslides in folium. We will need to replace this to match function in cell above
verified_locations = landslide_gdf[["lat", "lon", "type"]]

mp = folium.Map(
    location=[verified_locations.lat.mean(), verified_locations.lon.mean()],
    zoom_start=4,
    control_scale=True,
    tiles="Stamen Terrain")
for index, location_info in landslide_gdf.iterrows():
    folium.Marker([location_info["lat"], location_info["lon"]],
                  popup=location_info["type"],
                  icon=folium.Icon(color='red',
                                   prefix='fa',
                                   icon='exclamation-triangle')).add_to(mp)

display(mp)

## Establish AOI

In [18]:
# # Big Sur site id #9734
geoJSON = os.path.join('inputs', 'big_sur_siteid_9734.json')

# Wyoming site id #9806
# geoJSON = os.path.join('inputs', 'wyoming_siteid_9806.json')

In [8]:
# open geoJSON the houses polygon coordinates
with open(geoJSON, encoding='utf-8') as f:
    geoJSON = json.load(f)

# define coordinates
coords = geoJSON['features'][0]['geometry']['coordinates']
# create our aoi polygon from coords
aoi = ee.Geometry.Polygon(coords)

In [9]:
## can we move this lower? 
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds Earth Engine layers to a folium map."""
    
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

## Test new image_search function

In [10]:
image_search(landslide_gdf)

Enter verification code: 4/1AX4XfWiEiJOgvaOFczHIKmkDUBvlrHUeJhCRC4YdL0uPOiKpq-Jzv4YQCp8

Successfully saved authorization token.


EEException: Invalid argument specified for ee.Date(): 0      2015-09-21
1      2015-12-07
2      2015-10-04
3      2015-10-08
4      2015-12-09
          ...    
223    2017-07-11
224    2017-01-30
225    2017-04-18
226    2016-12-25
227    2017-02-28
Name: slide.date, Length: 228, dtype: object

![Sentinal-1](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSnEbm9fwTfAgz9UTlwD8_CD59LawTUhN2maA&usqp=CAU)
# Introducing Sentinal-1 Radar Imagery

SENTINEL-1 is an imaging radar mission providing continuous all-weather, day-and-night imagery at C-band. The SENTINEL-1 constellation provides high reliability, improved revisit time, geographical coverage and rapid data dissemination to support operational applications in the priority areas of marine monitoring, land monitoring and emergency services [1].

In this notebook we utilize the [COPERNICUS/S1_GRD_FLOAT](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD#bands) Earth Engine image collection. 

## Sattelite Image Collection and Preparation 
Below is a function that iterates through our geodataframe and collections images for each site location. We then have a function that will convert the image collection to a list and, clip the images to our AOI. 

The change detection will be more accurate if we utilize the 'VV' and the 'VH' images. We also include all of the images in the time series—this will ensure that we can reduce noise so we can be more certain that the change we are detecting is due to the landslide and not other background noise. 

In [ ]:
## Function that we will hopefully keep. 
# def image_search(gdf):
#     """
#     Iterates through a geoDataFrame and searches Google Earth Engine
#     based on certain parameters within the geoDataFrame. Buffers central
#     geometery search point by 1000 meters, event date by 180 days reciprocal. 

    
#     Parameters
#     ----------
#     data_file : geodataframe

#     Returns
#     -------
#     dataframe
#         appended with imagery timestamplist
    
#     """

#     # Set empty list
#     results = []
    

#     BASE_DATE = ee.Date(landslide_gdf['slide.date'])

#     # Data search in Google Earth Engine
#     im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
#                .filterBounds(gdf['geometry'])
#                .filter(ee.Geometry.Point.buffer({'distance': 1000}))
#                .filterDate(BASE_DATE, 
#                            BASE_DATE.advance(-180, 'days'), 
#                            BASE_DATE.advance(180, 'days'))
# #                .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
#                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
#                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
#                .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
#                .sort('date'))
#     timestamplist = (im_coll.aggregate_array('date')
#                             .map(lambda d: ee.String('T').cat(ee.String(d)))
#                             .getInfo())
    
#     # Add new column to geopandas DataFrame for avaiable data
#     for value in timestamplist:
#         if value == True:
#             results.append(timestamplist)
#         else:
#             results.append('No imagery available')

#     return gdf

## Big Sur Image Collection (hopefully can delete this with our function above)

In [11]:
im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
           .filterBounds(aoi)
           .filterDate(ee.Date('2016-09-01'), ee.Date('2017-09-01'))
           # Make sure the image properties are uniform
           .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
           .filter(ee.Filter.eq('relativeOrbitNumber_start', 42))
#            # Only keep dual-polarization configurations
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation',
                                          'VV'))
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation',
                                          'VH'))
           .map(lambda img: img.set('date',
                                    ee.Date(img.date()).format('YYYYMMdd')))
           .sort('date'))

timestamplist = (im_coll.aggregate_array('date')
                 .map(lambda d: ee.String('T').cat(ee.String(d)))
                 .getInfo())
timestamplist

['T20160915',
 'T20161009',
 'T20161102',
 'T20161126',
 'T20161220',
 'T20170113',
 'T20170119',
 'T20170206',
 'T20170218',
 'T20170302',
 'T20170326',
 'T20170407',
 'T20170419',
 'T20170501',
 'T20170513',
 'T20170606',
 'T20170618',
 'T20170630',
 'T20170712',
 'T20170724',
 'T20170805',
 'T20170817']

## Wyoming Image Collection (Hopefully can delete this with our function above)

In [12]:
# im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
#            .filterBounds(aoi)
#            .filterDate(ee.Date('2016-09-01'), ee.Date('2017-09-01'))
#            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
#            .filter(ee.Filter.eq('relativeOrbitNumber_start', 27))
#            .filter(ee.Filter.listContains('transmitterReceiverPolarisation',
#                                           'VV'))
#            .filter(ee.Filter.listContains('transmitterReceiverPolarisation',
#                                           'VH'))
#            .map(lambda img: img.set('date',
#                                     ee.Date(img.date()).format('YYYYMMdd')))
#            .sort('date'))

# timestamplist = (im_coll.aggregate_array('relativeOrbitNumber_start')
#                  .map(lambda d: ee.String('T').cat(ee.String(d)))
#                  .getInfo())
# timestamplist

### Convert and Clip Image Collection

In [13]:
def clip_img(img):
    """
    Clips a list of images to our aoi bounding box.

    Returns
    -------
    list
        clipped images to aoi
    
    """
    return ee.Image(img).clip(aoi)

im_list = im_coll.toList(im_coll.size())

# clip our list of images to the aoi bounding box
im_list = ee.List(im_list.map(clip_img))
im_list.get(0)
ee.Image(im_list.get(0)).bandNames().getInfo()
im_list.length().getInfo()

22

In [14]:
## Do not need this unless we want to try and make a folium map of site to show before and after. 
# def selectvv(current):
#     return ee.Image(current).select('VV')

# vv_list = im_list.map(selectvv)
# location = aoi.centroid().coordinates().getInfo()[::-1]
# rgb_images = (ee.Image.rgb(vv_list.get(9), vv_list.get(10), vv_list.get(11))
#               .log10().multiply(10))
# mp.add_ee_layer(rgb_images, {'min': -20,'max': 0}, 'rgb composite')
# mp.add_child(folium.LayerControl())

# Multitemporal Change Detection
In order to accurately detect land-change due to lanslides, we need to gather information across a time series (k) to determine the changes that occured and to get accurate location information for the landslides. 

We would like to know: when and where changes have taken place. In order to do this, we will highlight statistically significant changes through a series of "tests for change". We gathered in-depth statistical analyses from Earth Engine Python tutorials found [here](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-1), with an emphasis on the statistical changes outlined in [Part 3](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-3) by by Dr. Mort Canty (Canty, 2019). Please refer to the "References" section at the end of this notebook for the coppyright of their tutorial.

Our final product will be one map of a compilation of four different "change maps". Each respective change map represents the result of testing definite change in pixels at different intervals across the SAR images associate with our AOI. The following is the list of change maps:  

- cmap: the interval of the most recent change, one band, byte values ∈[0,k−1],
- smap: the interval of the first change, one band, byte values ∈[0,k−1],
- fmap: the number of changes, one band, byte values ∈[0,k−1],
- bmap: the changes in each interval,  k−1 bands, byte values ∈[0,1]).

**For the changes over our entire series of images, our null hypothesis is that, at a given pixel position, there has been no change in the signal strengths over the entire period.**


**The alternative hypothesis is that there was at least one change (and possibly many) over the interval.** 

In [15]:
def chi2cdf(chi2, df):
    """Calculates Chi square cumulative distribution function for
       df degrees of freedom using the built-in incomplete gamma
       function gammainc().
    """

    return ee.Image(chi2.divide(2)).gammainc(ee.Number(df).divide(2))

In [16]:
# Change map for alpha = 0.01.
k = len(timestamplist); alpha = 0.01
p_value = ee.Image.constant(1).subtract(chi2cdf(omnibus(vv_list), k-1))
c_map = p_value.multiply(0).where(p_value.lt(alpha), 1)
# Make the no-change pixels transparent.
c_map = c_map.updateMask(c_map.gt(0))

NameError: name 'omnibus' is not defined

In [17]:
# create a better docstring here. 
def sample_vv_imgs(j):
    """Samples the test statistics Rj in the region aoi."""
    j = ee.Number(j)
    # Get the factors in the expression for Rj.
    sj = vv_list.get(j.subtract(1))
    jfact = j.pow(j).divide(j.subtract(1).pow(j.subtract(1)))
    sumj = ee.ImageCollection(vv_list.slice(0, j)).reduce(ee.Reducer.sum())
    sumjm1 = ee.ImageCollection(vv_list.slice(0, j.subtract(1))).reduce(ee.Reducer.sum())
    # Put them together.
    Rj = sumjm1.pow(j.subtract(1)).multiply(sj).multiply(jfact).divide(sumj.pow(j)).pow(5)
    # Sample Rj.
    sample = (Rj.sample(region=aoi, scale=10, numPixels=1000, seed=123)
              .aggregate_array('VV_sum'))
    return sample

# Sample the first few list indices.
samples = ee.List.sequence(2, 5).map(sample_vv_imgs)

# Calculate and display the correlation matrix.
np.set_printoptions(precision=2, suppress=True)

NameError: name 'vv_list' is not defined

In [ ]:
def log_det_sum(im_list, j):
    """Returns log of determinant of the sum of the first j images in im_list."""
    im_ist = ee.List(im_list)
    sumj = ee.ImageCollection(im_list.slice(0, j)).reduce(ee.Reducer.sum())
    return ee.Image(det(sumj)).log()

def log_det(im_list, j):
    """Returns log of the determinant of the jth image in im_list."""
    im = ee.Image(ee.List(im_list).get(j.subtract(1)))
    return ee.Image(det(im)).log()

def pval(im_list, j, m=4.4):
    """Calculates -2logRj for im_list and returns P value and -2logRj."""
    im_list = ee.List(im_list)
    j = ee.Number(j)
    m2logRj = (log_det_sum(im_list, j.subtract(1))
               .multiply(j.subtract(1))
               .add(log_det(im_list, j))
               .add(ee.Number(2).multiply(j).multiply(j.log()))
               .subtract(ee.Number(2).multiply(j.subtract(1))
               .multiply(j.subtract(1).log()))
               .subtract(log_det_sum(im_list,j).multiply(j))
               .multiply(-2).multiply(m))
    pv = ee.Image.constant(1).subtract(chi2cdf(m2logRj, 2))
    return (pv, m2logRj)

def p_values(im_list):
    """Pre-calculates the P-value array for a list of images."""
    im_list = ee.List(im_list)
    k = im_list.length()

    def ells_map(ell):
        """Arranges calculation of pval for combinations of k and j."""
        ell = ee.Number(ell)
        # Slice the series from k-l+1 to k (image indices start from 0).
        im_list_ell = im_list.slice(k.subtract(ell), k)

        def js_map(j):
            """Applies pval calculation for combinations of k and j."""
            j = ee.Number(j)
            pv1, m2logRj1 = pval(im_list_ell, j)
            return ee.Feature(None, {'pv': pv1, 'm2logRj': m2logRj1})

        # Map over j=2,3,...,l.
        js = ee.List.sequence(2, ell)
        pv_m2logRj = ee.FeatureCollection(js.map(js_map))

        # Calculate m2logQl from collection of m2logRj images.
        m2logQl = ee.ImageCollection(pv_m2logRj.aggregate_array('m2logRj')).sum()
        pvQl = ee.Image.constant(1).subtract(chi2cdf(m2logQl, ell.subtract(1).multiply(2)))
        pvs = ee.List(pv_m2logRj.aggregate_array('pv')).add(pvQl)
        return pvs

    # Map over l = k to 2.
    ells = ee.List.sequence(k, 2, -1)
    pv_arr = ells.map(ells_map)

    # Return the P value array ell = k,...,2, j = 2,...,l.
    return pv_arr

#### Filtering the _P_ values

|Table 3.2 |       |       |       |       |       |        |
|----------|-------|-------|-------|-------|-------|--------|
|$i\ $ / $j$|      |     1 |     2 |     3 |     4 |        |
| 1        |       | $P_2$ | $P_3$ | $P_4$ | $P_5$ | $P_{Q5}$  |
| 2        |       |       | $P_2$ | $P_3$ | $P_4$ | $P_{Q4}$  |
| 3        |       |       |       | $P_2$ | $P_3$ | $P_{Q3}$  |
| 4        |       |       |       |       | $P_2$ | $P_{Q2}$  |

The pre-calculated _P_ values in _pv\_arr_ (shown schematically in Table 3.2 for $k=5$) are then scanned in nested iterations over indices $i$ and $j$ to determine the following thematic change maps:

- cmap: the interval of the most recent change, one band, byte values $\in [0,k-1]$,
- smap: the interval of the first change, one band, byte values $\in [0,k-1]$,
- fmap: the number of changes, one band, byte values $\in [0,k-1]$,
- bmap: the changes in each interval, $\ k-1$ bands, byte values $\in [0,1]$).

A boolean variable _median_ is included in the code. Its purpose is to reduce the salt-and-pepper effect in no-change regions, which is at least partly a consequence of the uniform distribution of the _P_ values under $H_0$ (see the section [A note on P values](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-2#a_note_on_p_values) in Part 2). If _median_ is _True_, the _P_ values for each $Q_\ell$ statistic are passed through a $5\times 5$ median filter before being compared with the significance threshold. This is not statistically kosher but probably justifiable if one is only interested in large homogeneous changes, for example flood inundations or deforestation.

Here is the code:

In [ ]:
def filter_j(current, prev):
    """Calculates change maps; iterates over j indices of pv_arr."""
    pv = ee.Image(current)
    prev = ee.Dictionary(prev)
    pvQ = ee.Image(prev.get('pvQ'))
    i = ee.Number(prev.get('i'))
    cmap = ee.Image(prev.get('cmap'))
    smap = ee.Image(prev.get('smap'))
    fmap = ee.Image(prev.get('fmap'))
    bmap = ee.Image(prev.get('bmap'))
    alpha = ee.Image(prev.get('alpha'))
    j = ee.Number(prev.get('j'))
    cmapj = cmap.multiply(0).add(i.add(j).subtract(1))
    # Check      Rj?            Ql?                  Row i?
    tst = pv.lt(alpha).And(pvQ.lt(alpha)).And(cmap.eq(i.subtract(1)))
    # Then update cmap...
    cmap = cmap.where(tst, cmapj)
    # ...and fmap...
    fmap = fmap.where(tst, fmap.add(1))
    # ...and smap only if in first row.
    smap = ee.Algorithms.If(i.eq(1), smap.where(tst, cmapj), smap)
    # Create bmap band and add it to bmap image.
    idx = i.add(j).subtract(2)
    tmp = bmap.select(idx)
    bname = bmap.bandNames().get(idx)
    tmp = tmp.where(tst, 1)
    tmp = tmp.rename([bname])
    bmap = bmap.addBands(tmp, [bname], True)
    return ee.Dictionary({'i': i, 'j': j.add(1), 'alpha': alpha, 'pvQ': pvQ,
                          'cmap': cmap, 'smap': smap, 'fmap': fmap, 'bmap':bmap})

def filter_i(current, prev):
    """Arranges calculation of change maps; iterates over row-indices of pv_arr."""
    current = ee.List(current)
    pvs = current.slice(0, -1 )
    pvQ = ee.Image(current.get(-1))
    prev = ee.Dictionary(prev)
    i = ee.Number(prev.get('i'))
    alpha = ee.Image(prev.get('alpha'))
    median = prev.get('median')
    # Filter Ql p value if desired.
    pvQ = ee.Algorithms.If(median, pvQ.focalMedian(2.5), pvQ)
    cmap = prev.get('cmap')
    smap = prev.get('smap')
    fmap = prev.get('fmap')
    bmap = prev.get('bmap')
    first = ee.Dictionary({'i': i, 'j': 1, 'alpha': alpha ,'pvQ': pvQ,
                           'cmap': cmap, 'smap': smap, 'fmap': fmap, 'bmap': bmap})
    result = ee.Dictionary(ee.List(pvs).iterate(filter_j, first))
    return ee.Dictionary({'i': i.add(1), 'alpha': alpha, 'median': median,
                          'cmap': result.get('cmap'), 'smap': result.get('smap'),
                          'fmap': result.get('fmap'), 'bmap': result.get('bmap')})

The following function ties the two steps together:

In [ ]:
def change_maps(im_list, median=False, alpha=0.01):
    """Calculates thematic change maps."""
    k = im_list.length()
    # Pre-calculate the P value array.
    pv_arr = ee.List(p_values(im_list))
    # Filter P values for change maps.
    cmap = ee.Image(im_list.get(0)).select(0).multiply(0)
    bmap = ee.Image.constant(ee.List.repeat(0, k.subtract(1))).add(cmap)
    alpha = ee.Image.constant(alpha)
    first = ee.Dictionary({'i': 1, 'alpha': alpha, 'median': median,
                           'cmap': cmap, 'smap': cmap, 'fmap': cmap, 'bmap': bmap})
    return ee.Dictionary(pv_arr.iterate(filter_i, first))

#### Post-processing: The Loewner order

The above change maps are still difficult to interpret. But what about _bmap_, the map of changes detected in each interval? Before we look at them it makes sense to include the direction of change, i.e., the [Loewner order](https://ieeexplore.ieee.org/document/8736751), see [Part 2](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-2#change_direction_the_loewner_order). In the event of significant change at time $j$, we can simply determine the positive or negative definiteness (or indefiniteness) of the difference between consecutive covariance matrix pixels

$$
c_j-c_{j-1},\quad j = 2,\dots,k,
$$

to get the change direction. But we can do better. Instead of subtracting the value for the preceding image, $c_{j-1}$, we can subtract the average over all values up to and including time $j-1$ for which no change has been signalled. For example for $k=5$, suppose there are significant changes in the first and fourth (last) interval. Then to get their directions we examine the differences

$$
c_2-c_1\quad{\rm and}\quad c_5 - (c_2+c_3+c_4)/3.
$$

The running averages can be conveniently determined with the so-called _provisional means algorithm_. The average $\bar c_i$ of the first $i$ images is calculated recursively as

$$
\begin{align*}
\bar c_i &= \bar c_{i-1} + (c_i - \bar c_{i-1})/i \cr
\bar c_1 &= c_1.
\end{align*}
$$

The function _dmap\_iter_ below is iterated over the bands of _bmap_, replacing the values for changed pixels with

- 1 for positive definite differences,
- 2 for negative definite differences,
- 3 for indefinite differences.

In [ ]:
def dmap_iter(current, prev):
    """Reclassifies values in directional change maps."""
    prev = ee.Dictionary(prev)
    j = ee.Number(prev.get('j'))
    image = ee.Image(current)
    avimg = ee.Image(prev.get('avimg'))
    diff = image.subtract(avimg)
    
    # Get positive/negative definiteness.
    posd = ee.Image(diff.select(0).gt(0).And(det(diff).gt(0)))
    negd = ee.Image(diff.select(0).lt(0).And(det(diff).gt(0)))
    bmap = ee.Image(prev.get('bmap'))
    bmapj = bmap.select(j)
    dmap = ee.Image.constant(ee.List.sequence(1, 3))
    bmapj = bmapj.where(bmapj, dmap.select(2))
    bmapj = bmapj.where(bmapj.And(posd), dmap.select(0))
    bmapj = bmapj.where(bmapj.And(negd), dmap.select(1))
    bmap = bmap.addBands(bmapj, overwrite=True)
    
    # Update avimg with provisional means.
    i = ee.Image(prev.get('i')).add(1)
    avimg = avimg.add(image.subtract(avimg).divide(i))
    # Reset avimg to current image and set i=1 if change occurred.
    avimg = avimg.where(bmapj, image)
    i = i.where(bmapj, 1)
    return ee.Dictionary({'avimg': avimg, 'bmap': bmap, 'j': j.add(1), 'i': i})

We only have to modify the _change\_maps_ function to include the change direction in the _bmap_ image:

In [ ]:
def change_maps(im_list, median=False, alpha=0.01):
    """Calculates thematic change maps."""
    k = im_list.length()
    # Pre-calculate the P value array.
    pv_arr = ee.List(p_values(im_list))
    # Filter P values for change maps.
    cmap = ee.Image(im_list.get(0)).select(0).multiply(0)
    bmap = ee.Image.constant(ee.List.repeat(0,k.subtract(1))).add(cmap)
    alpha = ee.Image.constant(alpha)
    first = ee.Dictionary({'i': 1, 'alpha': alpha, 'median': median,
                           'cmap': cmap, 'smap': cmap, 'fmap': cmap, 'bmap': bmap})
    result = ee.Dictionary(pv_arr.iterate(filter_i, first))
    
    # Post-process bmap for change direction.
    bmap =  ee.Image(result.get('bmap'))
    smap =  ee.Image(result.get('smap'))
    fmap =  ee.Image(result.get('fmap'))
    avimg = ee.Image(im_list.get(0))
    j = ee.Number(0)
    i = ee.Image.constant(1)
    first = ee.Dictionary({
        'avimg': avimg, 'bmap': bmap,'smap': smap, 'fmap': fmap,
        'j': j, 'i': i})
    dmap = ee.Dictionary(im_list.slice(1).iterate(dmap_iter, first)).get('bmap')
    return ee.Dictionary(result.set('bmap', dmap))

Here We export all of the change maps are exported as a single image with water mask.

In [ ]:
#test   How do we get the ndwi image for sentinal 1??????????***** this can possible change with im_search
slide_img = 'T20170606'
def plot_change_maps(im_list):
    """Compute and plot change maps"""

    # Run the algorithm with median filter and at 1% significance.
    result = ee.Dictionary(change_maps(im_list, median=True, alpha=0.01))

    # Extract the change maps and export to assets.
    cmap = ee.Image(result.get('cmap'))
    smap = ee.Image(result.get('smap'))
    fmap = ee.Image(result.get('fmap'))
    bmap = ee.Image(result.get('bmap'))
    cmaps = (
        ee.Image
        .cat(cmap, smap, fmap, bmap)
        .rename(['cmap', 'smap', 'fmap']+timestamplist[1:]))
    cmaps = cmaps.updateMask(cmaps.gt(0))
    location = aoi.centroid().coordinates().getInfo()[::-1]

    # Plot the result over Sentinel-2
    palette = ['black', 'red']
    params = {'min': 0, 'max': 1, 'palette': palette}
    
    Map = geemap.Map(location=location, zoom_start=15)
    
    #load image for ndwi
    image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')
    
    ndwi = image.normalizedDifference(['B3', 'B5'])
    ndwiViz = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}
    
    Map.addLayer(cmaps.select(slide_img),
                 {'min': 0, 'max': 1, 'palette': palette},
                 slide_img)
    Map.addLayer(ndwi, ndwiViz, 'NDWI', False)
    
    ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
    Map.addLayer(ndwiMasked, ndwiViz, 'NDWI masked')

    return Map

In [ ]:
plot_change_maps(im_list)

In [ ]:
# # Wyoming
# cmaps = cmaps.updateMask(cmaps.gt(0))

# location = aoi.centroid().coordinates().getInfo()[::-1]
# palette = ['black', 'red', 'cyan', 'yellow', 'grey']
# mp = folium.Map(location=location, zoom_start=15)
# slide_img = 'T20170217'
# cmaps.select(slide_img)
# mp.add_ee_layer(
#     cmaps.select(slide_img),
#     {'min': 0, 'max': 4, 'palette': palette},
#     slide_img)

# mp.add_child(folium.LayerControl())

# Summary (expand on this)

The definite (red) changes correspond to decreases and increases in intensity of _VV_ and _VH_ reflectance and are due to displaced land caused by landslides. Areas of definite change do correspond to locations identified on our verified landslide catalog.  

# Outlook (expand on this)

This study shows us that it is possible that SAR imagery can be utilized to detect landslide motion. One possible application is rapid detection. In this notebook we displayed each change map with the color red. It is possible to display the different individual change maps with different colors. This could help determine locations wtih motion pre-landslide (smap: or interval with first change), which could help inform areas of high landslide risk.

### References
Copyright 2022 Mitchel Thompson and Leah Manak

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

[1] https://sentinel.esa.int/web/sentinel/user-guides/sentinel-1-sar/